# Analyse des performances des algorithmes de résolution du taquin

Ce notebook analyse les performances de trois algorithmes de résolution du taquin :
- BFS (Breadth-First Search)
- DFS (Depth-First Search)
- A* (A-star)

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import signal
import psutil
import tracemalloc
from contextlib import contextmanager
from solver_npuzzle import solve_bfs, solve_dfs, solve_astar
from npuzzle import load_puzzle
from node import Node
from generate_npuzzle import generate_puzzles

## Configuration du timeout et suivi de la mémoire

In [21]:
@contextmanager
def timeout(seconds):
    def signal_handler(signum, frame):
        raise TimeoutError("Timeout!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

def get_memory_usage():
    process = psutil.Process()
    return process.memory_info().rss / 1024 / 1024  # Convertir en MB

## Génération des puzzles de test

Nous allons générer des puzzles de différentes tailles et difficultés pour tester nos algorithmes.

In [22]:
# Création du dossier pour les puzzles
if not os.path.exists('puzzles'):
    os.makedirs('puzzles')

# Génération des puzzles
sizes = [3, 4]  # Tailles des puzzles (3x3 et 4x4)
max_lengths = [5, 10, 15, 20, 25, 30, 35, 40]  # Nombre de mouvements pour mélanger
puzzles_per_length = 2  # Nombre de puzzles par longueur

for size in sizes:
    for length in max_lengths:
        generate_puzzles(size, length, puzzles_per_length, 'puzzles')

Saving puzzle to puzzles/npuzzle_3x3_len1_0.txt
Saving puzzle to puzzles/npuzzle_3x3_len1_1.txt
Saving puzzle to puzzles/npuzzle_3x3_len2_0.txt
Saving puzzle to puzzles/npuzzle_3x3_len2_1.txt
Saving puzzle to puzzles/npuzzle_3x3_len3_0.txt
Saving puzzle to puzzles/npuzzle_3x3_len3_1.txt
Saving puzzle to puzzles/npuzzle_3x3_len4_0.txt
Saving puzzle to puzzles/npuzzle_3x3_len4_1.txt
Saving puzzle to puzzles/npuzzle_3x3_len5_0.txt
Saving puzzle to puzzles/npuzzle_3x3_len5_1.txt
Saving puzzle to puzzles/npuzzle_3x3_len1_0.txt
Saving puzzle to puzzles/npuzzle_3x3_len1_1.txt
Saving puzzle to puzzles/npuzzle_3x3_len2_0.txt
Saving puzzle to puzzles/npuzzle_3x3_len2_1.txt
Saving puzzle to puzzles/npuzzle_3x3_len3_0.txt
Saving puzzle to puzzles/npuzzle_3x3_len3_1.txt
Saving puzzle to puzzles/npuzzle_3x3_len4_0.txt
Saving puzzle to puzzles/npuzzle_3x3_len4_1.txt
Saving puzzle to puzzles/npuzzle_3x3_len5_0.txt
Saving puzzle to puzzles/npuzzle_3x3_len5_1.txt
Saving puzzle to puzzles/npuzzle_3x3_len

## Visualisation des résultats

In [ ]:
# Supposons que tu as déjà une liste de fichiers puzzles
puzzle_files = sorted([f for f in os.listdir('puzzles') if f.endswith('.txt')])

results = []
for puzzle_file in puzzle_files:
    path = os.path.join('puzzles', puzzle_file)
    puzzle = load_puzzle(path)
    root = Node(state=puzzle, move=None)
    
    # 1. Calculer la difficulté réelle (longueur de la solution BFS)
    sol_bfs = solve_bfs([root])
    if not sol_bfs:
        continue  # Puzzle non résoluble, on saute
    difficulty = len(sol_bfs)
    
    # 2. Mesurer le temps pour chaque algo
    import time
    t0 = time.time()
    solve_bfs([root])
    t_bfs = time.time() - t0

    t0 = time.time()
    solve_dfs([root])
    t_dfs = time.time() - t0

    t0 = time.time()
    solve_astar([root], [])
    t_astar = time.time() - t0

    results.append((difficulty, t_bfs, t_dfs, t_astar))

# 3. Trier les résultats par difficulté croissante
results.sort(key=lambda x: x[0])

# 4. Tracer les courbes
difficulties = [r[0] for r in results]
bfs_times = [r[1] for r in results]
dfs_times = [r[2] for r in results]
astar_times = [r[3] for r in results]

plt.figure(figsize=(10,6))
plt.plot(difficulties, bfs_times, label='BFS')
plt.plot(difficulties, dfs_times, label='DFS')
plt.plot(difficulties, astar_times, label='A*')
plt.xlabel("Difficulté (longueur de la solution BFS)")
plt.ylabel("Temps de résolution (secondes)")
plt.title("Comparaison des algorithmes sur le taquin")
plt.legend()
plt.grid(True)
plt.show()